In [1]:
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.metrics import RMSE
from cornac.models import  ItemKNN, UserKNN, NMF, BPR, MF,VAECF, NeuMF,PMF, WMF
# from MF import MF
import pandas as pd
import numpy as np
import cornac
import math
# import seaborn as sns
# import matplotlib.pyplot as plt

/Users/tahsinalamgirkheya/anaconda3/envs/re-ranking/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rating_data_pd = pd.read_csv(
    "../data/bx/indexed_interactions_n.csv",
    sep="\t",
    header=0,
    # names=["userID", "itemID", "Rating", "Timestamp"],
)
rating_data = rating_data_pd.to_numpy()
rating_data.__len__()
rating_data_pd


,ISBN,genre,userID,MovieID,Rating,Age
0,3426029553,fiction|historical fiction,0,0,8,27.0
1,3426029553,fiction|historical fiction,1,0,8,32.0
2,3426029553,fiction|historical fiction,2,0,9,33.0
3,3426029553,fiction|historical fiction,3,0,10,35.0
4,3426029553,fiction|historical fiction,4,0,8,39.0
...,...,...,...,...,...,...
83956,0399147020,mystery|fiction,599,9046,8,42.0
83957,0821772341,fiction|romance|self-help,1381,9047,8,47.0
83958,0399146180,mystery|fiction,1901,9048,7,69.0
83959,0399146180,mystery|fiction,4688,9048,7,55.0


In [3]:
books = rating_data_pd[["MovieID", "ISBN", "genre"]].drop_duplicates()
books

,MovieID,ISBN,genre
0,0,3426029553,fiction|historical fiction
6,1,3551551677,children's books|fantasy|historical fiction|ho...
12,2,3551551685,children's books|fantasy|drama|fiction|educati...
18,3,3551551693,children's books|fantasy|drama|fiction|travel|...
23,4,3551551936,children's books|fantasy|drama|fiction|educati...
...,...,...,...
83954,9045,0688168248,fiction|drama
83956,9046,0399147020,mystery|fiction
83957,9047,0821772341,fiction|romance|self-help
83958,9048,0399146180,mystery|fiction


In [5]:
unique_categories = ['fiction',
 'non-fiction',
 'fantasy',
 'science fiction (sci-fi)',
 'mystery',
 'thriller',
 'horror',
 'romance',
 'historical fiction',
 'biography & autobiography',
 'self-help',
 'young adult (ya)',
 "children's books",
 'classics',
 'poetry',
 'drama',
 'graphic novels & comics',
 'travel',
 'spirituality & religion',
 'educational & academic']

for c in unique_categories:
    books[c] = 0
for index, row in books.iterrows():
    cats = row["genre"].split("|")
    for cat in cats:
        books.at[index, cat] = 1
books

,MovieID,ISBN,genre,fiction,non-fiction,fantasy,science fiction (sci-fi),mystery,thriller,horror,...,self-help,young adult (ya),children's books,classics,poetry,drama,graphic novels & comics,travel,spirituality & religion,educational & academic
0,0,3426029553,fiction|historical fiction,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,3551551677,children's books|fantasy|historical fiction|ho...,1,0,1,1,1,0,1,...,0,0,1,0,0,1,0,0,0,1
12,2,3551551685,children's books|fantasy|drama|fiction|educati...,1,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
18,3,3551551693,children's books|fantasy|drama|fiction|travel|...,1,0,1,1,0,0,0,...,0,0,1,0,0,1,0,1,0,1
23,4,3551551936,children's books|fantasy|drama|fiction|educati...,1,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83954,9045,0688168248,fiction|drama,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
83956,9046,0399147020,mystery|fiction,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
83957,9047,0821772341,fiction|romance|self-help,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
83958,9048,0399146180,mystery|fiction,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
users = rating_data_pd[["userID", "Age"]].drop_duplicates()
users

,userID,Age
0,0,27.0
1,1,32.0
2,2,33.0
3,3,35.0
4,4,39.0
...,...,...
68178,4977,25.0
68960,4978,32.0
70444,4979,41.0
71198,4980,38.0


In [11]:
dataset = rating_data_pd[["userID", "MovieID", "Rating"]].to_numpy()
dataset

array([[   0,    0,    8],
       [   1,    0,    8],
       [   2,    0,    9],
       ...,
       [1901, 9048,    7],
       [4688, 9048,    7],
       [ 795, 9049,    5]])

In [12]:
rating_data_pd

,ISBN,genre,userID,MovieID,Rating,Age
0,3426029553,fiction|historical fiction,0,0,8,27.0
1,3426029553,fiction|historical fiction,1,0,8,32.0
2,3426029553,fiction|historical fiction,2,0,9,33.0
3,3426029553,fiction|historical fiction,3,0,10,35.0
4,3426029553,fiction|historical fiction,4,0,8,39.0
...,...,...,...,...,...,...
83956,0399147020,mystery|fiction,599,9046,8,42.0
83957,0821772341,fiction|romance|self-help,1381,9047,8,47.0
83958,0399146180,mystery|fiction,1901,9048,7,69.0
83959,0399146180,mystery|fiction,4688,9048,7,55.0


In [19]:
ratio_split = StratifiedSplit(
    data=dataset,
    test_size=0.2,
    rating_threshold=0,
    # val_size=0.1,
    seed=123,
    verbose=True,
    fmt="UIR",
)
hr_10 = cornac.metrics.HitRatio(k=20)
ndcg_10 = cornac.metrics.NDCG(k=20)
recall_10 = cornac.metrics.Recall(k=20)
prec_10 = cornac.metrics.Precision(k=20)
auc = cornac.metrics.AUC()
map = cornac.metrics.MAP()

epochs = [20]
models = []
for i in range(len(epochs)):
    models.append(
        NeuMF(
           name=f"NeuMFe={epochs[i]}",
            num_factors=16,
            layers=( 32, 16),
            act_fn="relu",
            reg=0.0,
            num_epochs=epochs[i],
            batch_size=256,
            num_neg=3,
            lr=0.0001,
            learner="adam",
            backend="tensorflow",
            early_stopping=None,
            trainable=True,
            verbose=True,
            seed=123,
        )
    )
    
cornac.Experiment(
    ratio_split, models=models, metrics=[hr_10, ndcg_10, recall_10, auc, prec_10, map]
).run()

rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 4982
Number of items = 8998
Number of ratings = 65260
Max rating = 10.0
Min rating = 1.0
Global mean = 7.9
---
Test data:
Number of users = 4982
Number of items = 8998
Number of ratings = 18597
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 4982
Total items = 8998

[NeuMFe=20] Training started!


100%|██████████| 20/20 [00:22<00:00,  1.12s/it, loss=0.485]



[NeuMFe=20] Evaluation started!


Ranking: 100%|██████████| 4977/4977 [00:09<00:00, 497.85it/s]


TEST:
...
          |    AUC | HitRatio@20 |    MAP | NDCG@20 | Precision@20 | Recall@20 | Train (s) | Test (s)
--------- + ------ + ----------- + ------ + ------- + ------------ + --------- + --------- + --------
NeuMFe=20 | 0.6796 |      0.1157 | 0.0133 |  0.0213 |       0.0064 |    0.0382 |   22.5607 |  10.0022

